# Process data

### Separate threads and posts

In [ ]:
import csv
import sys
import codecs
import os
import re
from difflib import SequenceMatcher
import itertools
import json
import codecs
import pandas as pd

#this is to extent the size of the reading csv cell
# csv.field_size_limit(sys.maxsize)
file_path='path_to_raw+data'
save_path_trash='path_to_save_data'


def UTC_separation(thread):
    posts_lst=[]
    posts=re.split(r'(\d\d:\d\d, \d+ \w+ \d\d\d\d \(UTC\))',thread)# split the posts based on the timestamp
    for i in range(0,len(posts)-1, 2):
        posts_lst.append(posts[i]+posts[i+1])# connects post with timestamp because the splot separates them
    
    return posts_lst

#to separate discussions with no title
def thread_without_title(temp, page_title, save_path_data):
    if ((temp[0:2]=='{{') and (('documentation' in temp) or ('Documentation' in temp))) or ((temp[0:2]=='{{') and (('Interwiki conflict' in temp) or ('interwiki conflict' in temp))) or ('#REDIRECT' in temp): # checks if the text includes only documentation, not discussions
         trash=pd.DataFrame({'page_title':[page_title], 'text':temp})
         trash.to_csv(str(save_path_trash),mode='a',index=False, header=not os.path.exists(str(save_path_trash)))
    else:
        thread_title='No subject'
        posts_lst = UTC_separation(temp)
        #store posts with the corresponding sibject
        df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
        #save to csv 
        df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))


def thread_with_title(titles, temp, page_title, save_path_data):
    for j in range(len(temp)):
        if (temp[j] in titles):
            if (j+1)==(len(temp)):
                thread_title=temp[j]
                posts_lst = []
                    #store posts with the corresponding sibject
                df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
                    #save to csv 
                df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))
            else:
                thread_title=temp[j]
                posts_lst = UTC_separation(temp[j+1])
                    #store posts with the corresponding sibject
                df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
                    #save to csv 
                df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))
        
    

#function to create a file with the separated posts  
def separate_discussions(text, page_title, save_path_data):

    titles = re.findall('==(.*)==', text)# find the titles in the discussion  
    titles=[s.replace('=', '') for s in titles]# removes unecessery = in the title
    
    
    #titles=list(set(titles)) 
    temp= text.split('==') # split the text based on == , includided in the title
    temp = [x for x in temp if x != ''] # stores the splited parts
    temp=[s.strip('=') for s in temp]# removes unecessery = in the title

    # ---Step 1: there is no title in the page
    if titles==[]: # check if there is title in the begining of the page
        thread_without_title(temp[0], page_title, save_path_data) 
    else:
        if temp[0] not in titles:
            thread_without_title(temp[0], page_title, save_path_data)# ---Step 2: the first thread do not have title 
            thread_with_title(titles, temp, page_title, save_path_data)# ignores the first if does not have a title and process the rest threads
        else:
            thread_with_title(titles, temp, page_title, save_path_data)# ---Step 3: a;; threads have titles 



file_lst=['items.csv','properties.csv']
    

for file in file_lst:
    save_path_data='path'+str(file) # path to save posts and threads
    data=pd.read_csv(str(file_path)+str(file), encoding='utf-8')# read file
    for row in range(len(data)):# read row by row (i.e., page by page)
        print(row)
        # print(data.loc[row,'page_title'])
        # print(data.loc[row,'text'])
        page_title=data.loc[row,'page_title']
        if isinstance(data.loc[row,'text'],str):# check of the row is nan. If it is nan the type is float not str
            separate_discussions(data.loc[row,'text'], page_title, save_path_data)
        else:
            empty=pd.DataFrame({'page_title':[page_title], 'text':'empty'})
            empty.to_csv(str(save_path_trash),mode='a',index=False, header=not os.path.exists(str(save_path_trash)))

### Extract username and timestamp

In [ ]:
import pandas as pd
import os
import re

# find usernmae based on [[User talk:]] pattern
def find_username_User_talk(text):
    pattern=re.compile(r"(\[)?(User:|User talk:|user:|user talk:|User Talk:|User_talk:|User_Talk:|user_talk:|Utente:|Usuário:|Utilisateur:|Utilizator:|Usuari Discussió:|Usuario discusión:|Usuario:|Usuario Discusi\u00f3n:|kullanıcı:|Kullan\u0131c\u0131:|:USER TALK:|:USER:|U:|u:|Benutzer:|Benutzer Diskussion:|Special:Contributions/|welcominguser=|Discussão:|Dyskusja_Wikipedysty:|사용자토론:)(.*?)(\||/|\])") # the regular expression I need to extract 
    usernames=pattern.finditer(text)
    #search for the username
    names=[]        
    for match in usernames:
        #print(match.group(2))
        names.append([match.group(3),match.end()])
        
    if len(names)>0:
        max_loc=max([sublist[1] for sublist in names])
        name=[sublist[0] for sublist in names if sublist[1]==max_loc]
    else: name=[]
    return name




#funstion to extract username from the post
def extract_info(raw_text, anon):
    timestamp='No date'
    username= 'Unsigned post'
    text1=raw_text.replace('‎','')
    t_split=text1.split('(UTC)') # this is mainly for property proposal, because after (UTC) it includes remaining punctuation
    t_connect1=[str(part)+'(UTC)' for part in t_split[ :-1]] # remove the final part with punctuaton
    t_connect='(UTC)'.join(t_connect1) # connect again the text
    
    if t_connect.endswith('(UTC)'):# if UTC in not in the text, the post does not have a signature or timestmap so the usernmae will be assigned as unsigned and the timestamp with no date
        peaces=re.split("(\d+:\d\d, \d+ \w+ \d\d\d\d \(UTC\))$", t_connect)
        if len(peaces)>1:
            timestamp=peaces[1]
            u_user_talk=find_username_User_talk(peaces[0])# step 1--- detect the pattern [[user talk:]] to find the username
            
            if len(u_user_talk)<1:# if step 1 do not work
                u_from_list= [name for name in username_lst if name in peaces[0]]# step 2 ----- check the usernmae list
                
                for name in u_from_list:
                    text=peaces[0].replace('(talk) ','')
                    u_peaces=re.split(name,text)
                    if len(u_peaces[-1])<10:
                        print(name)
                        return timestamp, name, anon
                if len(u_from_list)<1 or username== 'Unsigned post':# if step 1 and 2 do not work the username will be assigned as anonymous
                    anon += 1
                    username= 'Anonymous_username_'+str(anon)
                    print(username)
                    return timestamp, username, anon
                    
            else:
                print(u_user_talk)
                return timestamp, u_user_talk[0], anon
        else:
            print(username)
            return timestamp, username, anon    
    else:
        print(username)
        return timestamp, username, anon

    

 #---------RUN-----------


file_path='path_to_directory__files_for_process'
save_path='path_to_directory_to_save_files'

# load the list with usernames we have gathered during the dump process 
# read usernames 
my_file = open("path_to_username_list", "r", encoding='utf-8')
data = my_file.read()
# replacing end splitting the text 
# when newline ('\n') is seen.
username_lst = data.split("\n")
my_file.close()
# #separate usernmaes based on length because there are usernmaes with one character and confuse the correct detection
# usernames_short=[n for n in username_lst if len(n)<=2]
# usernames_long=[n for n in username_lst if len(n)>3]




anon=0 # this is to give names in case we can not recognise a username
for dump in os.listdir(str(file_path)):
    # dump='property_proposal_all.csv'
    df=pd.read_csv(str(file_path)+str(dump), encoding='utf-8')
    for row in range(len(df)):
        print(dump, row)
        # test="=== {{TranslateThis | anchor = en| en = Kinsky–Halm Catalogue <!-- |xx = property names in some other languages -->}} ==={{Property proposal|status			= 7161|description		= {{TranslateThis | en = Number in the Kinsky–Halm Catalogue of the works of Ludwig van Beethoven, which do not have opus numbers, or are fragmentary.<!-- | xx = descriptions in other languages --> }}|subject item           = {{Q|616043}}|infobox parameter	= [[en:catalogue:infobox musical composition]]<!-- Wikipedia infobox parameters, if any; ex: "population" in [[:en:template:infobox settlement]] -->|datatype		= external-id<!-- put datatype here (one of: item, external-id, string, media, coordinates, monolingual text, multilingual text, time, URL, number, geo-shape, tabular, lexeme, form, sense, mathematical expression, or a possibly new one) -->|domain			= <!-- entity type (item, property, lexeme, form, sense) and types of items that may bear this property; give QIDs if possible -->|allowed values		= \d+<!-- type of linked items (Q template or text), string pattern (regex if possible), list or range of allowed values, etc. -->|allowed units          = <!-- units that are allowed for values of this property -->|source			= https://en.wikipedia.org/wiki/WoO <!-- external reference URL, Wikipedia list article, etc. --><!-- you should provide 3 examples at least-->|example 1		= {{Q|8076022}} → 123 <!-- {{Q|1}} → value -->|example 2		= {{Q|166550}} → 4|example 3		= {{Q|166173}} → 6|example 4		= <!--  if convenient, include more examples by following the same pattern: |example ...		=  {{Q|1}} → value-->|planned use            = Add to Authority control<!-- what use do you plan to make of the property within the next month -->|number of ids          = <!-- for identifiers, the number of ids available from the source -->|expected completeness  = Q21873974 <!-- for identifiers, the coverage we can hope for in Wikidata: Q21873886 (always incomplete) or Q21873974 (eventually complete) -->|formatter URL		= <!-- for identifiers, URL pattern where $​1 replaces the value -->|external links         = <!-- for identifiers, search string to pass to sister projects' Special:LinkSearch pages, e.g. example.com  -->|see also               = <!-- other, related properties -->|filter			= <!-- sample: 7 digit number can be validated with edit filter [[Special:AbuseFilter/17]] -->|robot and gadget jobs	= <!-- Should or are bots or gadgets doing any task with this? (Checking other properties for consistency, collecting data, etc.) -->|subpage		= Werke ohne Opuszahl|topic			= authority control}}====Motivation====This is an important catalogue, which we should be able to reference, possibly as authority control.  I leave it to more experienced folk to consider whether there should be a generic WoO and a separate Kinsky–Halm Catalogue property. (E.G. http://www.raff.org/music/catalog/opus/woo.htm is a WoO catalogue for Joachim Raff.)All&nbsp;the&nbsp;best: ''[[Q23041480|Rich]]&nbsp;[[User talk:Rich Farmbrough|Farmbrough]]'',&nbsp;<small>12:41,&nbsp;4&nbsp;August&nbsp;2019&nbsp;(UTC).</small><br />===="
        # timestamp_row, username_row, anon = extract_info(test, anon)
        timestamp_row, username_row, anon = extract_info(df.loc[row,'post'], anon)
        save_df=pd.DataFrame({'post':[df.loc[row,'post']],'page_title':[df.loc[row,'page_title']],'thread_title':[df.loc[row,'thread_title']],'urername':[username_row],'timestamp':[timestamp_row]})
        save_df.to_csv(str(save_path)+str(dump),mode='a',index=False, header=not os.path.exists(str(save_path)+str(dump)))

### Asign IDs to usernames, posts, threads, and pages

In [ ]:
# Usernames
import pandas as pd

df=pd.read_csv('path_to_file_with_username_inflo', encoding='utf-8')
df_id=pd.DataFrame(columns=['UID','username','user_type','registration','rights','asking','responding','num_posts', 'talking_type'])
for row in range(len(df)):
    df_id.loc[row,'UID']='U'+str(row)
    df_id.iloc[row,1:2]=df.iloc[row,2:3]
    df_id.iloc[row,3:]=df.iloc[row,5:]
df_id.to_csv("path_to_save",encoding='utf-8',index=False)

In [ ]:
# post, thread, and page ids

import pandas as pd
import os

UID_df=pd.read_csv("username_ids.csv",encoding='utf-8')

path='path_to_directory_with_post_info'

p_id=0
pt_id=0
t_id=0
for file in os.listdir(str(path)):

    # file='items.csv'
    df=pd.read_csv(str(path)+str(file),encoding='utf-8')

    # dropping the rows having NaN values
    df = df.dropna()
    # To reset the indices
    df = df.reset_index(drop=True)

    df_id=pd.DataFrame(columns=['PID','post','PTID','page_title','TID','thread_title','UID','username','timestamp','answer2post'])

    # set page_title ids
    pt_uniqies=df['page_title'].unique()
    pt_ids_df=None
    pt_ids_df=pd.DataFrame({'title':pt_uniqies,'id':['PT'+str(x) for x in range(pt_id,pt_id+len(pt_uniqies))]})
    pt_id=pt_id+len(pt_uniqies)

    #set thread_title ids
    t_ids_df=pd.DataFrame()
    for pt in pt_uniqies:
        df_pt=df.loc[df['page_title']==pt]
        t_uniqies=df_pt['thread_title'].unique()
        t_ids_df1=None
        t_ids_df1=pd.DataFrame({'title':t_uniqies,'id':['T'+str(x) for x in range(t_id,t_id+len(t_uniqies))], 'page':[pt]*len(t_uniqies)})
        t_id=t_id+len(t_uniqies)
        t_ids_df=pd.concat([t_ids_df,t_ids_df1])

    for row in range(len(df)):
        print(file, row)
        df_id.loc[row,'PID']='P'+str(p_id+row)
        df_id.loc[row,'post']=df.loc[row,'post']
        df_id.loc[row,'PTID']=pt_ids_df.loc[pt_ids_df['title']==df.loc[row,'page_title'],'id'].values[0]
        df_id.loc[row,'page_title']=df.loc[row,'page_title']
        df_id.loc[row,'TID']=t_ids_df.loc[((t_ids_df['title']==df.loc[row,'thread_title'])&(t_ids_df['page']==df.loc[row,'page_title'])),'id'].values[0]
        df_id.loc[row,'thread_title']=df.loc[row,'thread_title']
        df_id.loc[row,'UID']=UID_df.loc[UID_df['username']==df.loc[row,'urername'],'UID'].values[0]
        df_id.loc[row,'username']=df.loc[row,'urername']
        df_id.loc[row,'timestamp']=df.loc[row,'timestamp']
        df_id.loc[row,'answer2post']=df.loc[row,'answer2post']
        


        # df_id.iloc[row,1:2]=df.iloc[row,2:3]
        # df_id.iloc[row,3:]=df.iloc[row,5:]

    p_id=p_id+row+1
    df_id.to_csv('path_to_save_directory'+str(file), encoding='utf-8', index=False)

## Editors' features

### Registration date

We used a Wikidata API https://www.wikidata.org/w/api.php?action=help&modules=query%2Busers

In [ ]:
#extract the info

import requests
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

S = requests.Session()

URL = "https://www.wikidata.org/w/api.php"#info about the features you can extract https://www.wikidata.org/w/api.php?action=help&modules=query%2Busers


editors_count=0
line_count=0


users = pd.DataFrame(columns=['userid','name','editcount', 'registration','rights','name_post'])#create a data frame with the features you choose

editors_count=0
#path to editors names
with open('known_names_APIinput.txt', encoding="utf-8") as f:
    lines = f.readlines()
    for l in lines:
        
        #editors_in_each_line_count=0
        for part in l.split("|"):
            editors_count += 1
            print('count: ',editors_count, " editor: ", part)
            # line_count+=1
            #editors_in_each_line_count+=1
            
            PARAMS = {
            "action": "query",
            "format": "json",
            "list": "users",
            "ususers": part,
            # "usprop": "blockinfo|editcount|registration|rights"} #you choose features to extarct
            "usprop": "editcount|registration|rights"} #you choose features to extarct
            

            
            R = S.get(url=URL, params=PARAMS)
            DATA = R.json()
            
            USERS = DATA["query"]["users"]
            # print(USERS[0])
            row=USERS[0]
            row['name_post']=part  
            users = users.append(row, ignore_index=True)    
            #unknown_users=pd.DataFrame(columns=['names'])  
            # for u in USERS:
            #     editors_count+=1
            #     #if ('invalid' not in u.keys()) and ('missing' not in u.keys()):
            #         #print(str(u["name"]) + " has " + str(u["editcount"]) + " edits.")
            #         #print(str(u["name"]) + ' with id ' + str(u['userid']))
            #         #print(u['groupmemberships'])
            #     users = users.append(u, ignore_index=True)
            #     #else:unknown_users=unknown_users.append(u, ignore_index=True)
                
            
            #path to save       
            users.to_csv('known_names_info_output_v2.csv', encoding='utf-8', index=False)
            #unknown_users.to_csv('/mnt/data/elisavetk/Theme_2/Post_Graph/unknown_editor_info.csv', encoding='utf-8', index=False,  mode='a', header=False)

#print(editors_count)

'''if editors_in_each_line_count == 49:
    break'''
#break

### Editors' type 

registered, unregistered, unknown names, bots

In [ ]:
import pandas as pd
import re

#load usernames found in discussions
df=pd.read_csv('unique_usernames_found_in_discussions.csv', encoding='utf-8')
u_df_lst=df['username'].tolist()

# store known usernames. it is the merge of the above two
u_common=df

# store anonymou_usernames
u_anonymous=[name for name in u_df_lst if 'Anonymous_username_'in str(name)]

# store Unregistered users
pattern1=r"[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}"
pattern2=r"(([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,7}:|([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2}|([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3}|([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4}|([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))"
u_IP4=[name for name in u_df_lst if bool(re.match(pattern1,str(name)))]
u_IP6=[name for name in u_df_lst if bool(re.match(pattern2,str(name)))]


# len(u_common)+len(u_anonymous)+len(u_IP4)+len(u_IP6)
known_names=u_common['username'].to_list()+u_anonymous+u_IP4+u_IP6 # registered and unregistered editors
unknown_names=[name for name in u_df_lst if str(name) not in known_names] # unknwon names



#check how many names are unknown names
len(unknown_names)*100/len(u_df_lst)
# the uknown names are 5% of the total names I found in discussions

# create df with the username info for the three files
u_data1_df=data(known_names)
# u_data2_df=data(u_data2)
u_data3_df=data(unknown_names)

# store data anonymou_usernames
u_anonymous=[name for name in df['username'] if 'Anonymous_username_'in str(name)]
u_anonymous_data={name:[None,name,'anonymous_username',None, None, None] for name in u_anonymous}
u_anonymous_df = pd.DataFrame([v for k, v in u_anonymous_data.items()], columns=['userid', 'name', 'username_type', 'editcount_API','registration','rights'])


# store data Unregistered users
pattern1=r"[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}"
pattern2=r"(([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,7}:|([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2}|([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3}|([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4}|([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))"
u_IP4=[name for name in df['username'] if bool(re.match(pattern1,str(name)))]
u_IP4_data={name:[None,name,'unregistered',None, None, None] for name in u_IP4}
u_IP4_df = pd.DataFrame([v for k, v in u_IP4_data.items()], columns=['userid', 'name', 'username_type', 'editcount_API','registration','rights'])

u_IP6=[name for name in df['username'] if bool(re.match(pattern2,str(name)))]
u_IP6_data={name:[None,name,'unregistered',None, None, None] for name in u_IP6}
u_IP6_df = pd.DataFrame([v for k, v in u_IP6_data.items()], columns=['userid', 'name', 'username_type', 'editcount_API','registration','rights'])


# df_full=pd.concat([u_data1_df,u_data2_df,u_data3_df,u_anonymous_df,u_IP4_df,u_IP6_df])
df_full=pd.concat([u_data1_df,u_data3_df,u_anonymous_df,u_IP4_df,u_IP6_df])
df_full.to_csv('username_type.csv',encoding='utf-8',index=None)


In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("username_type.csv", encoding='utf-8')
df_bots=pd.read_csv('bot_names.csv', encoding='utf-8')
bot_name=df_bots['Name'].to_list()

for i in range(len(df['username_type'])):
     if df.loc[i,'name'] in bot_name:
          df.loc[i,'username_type']='bot'

df.to_csv('username_type.csv', encoding='utf-8',index=False)

### Editors' edit history

Process edit history to measure account age (the difference between registare date and last edit), max gaps of inacitvity (last edit and today), and number of edits

In [ ]:
import pandas as pd
import os
import numpy as np
from os import listdir
from os.path import isfile, join


directory_path='username_edits/'# these are files with (username, timestamp) for editors' edits

# last timestamp for the archived data
today=pd.to_datetime('2023-03-02T00:00:00Z', format='%Y-%m-%dT%H:%M:%SZ')


#load username information
all_users=pd.read_csv('username_info_v2.csv',encoding='utf-8')
#keep registered users
#create a lsit with registered editors
registered=all_users.loc[all_users['username_type'] == 'registered']
# registered=registered1.head(1000)

#lists to store data
last_edit=[]
acount_age=[]
max_gap=[]
last_today=[]
number_of_edits=[]
false_edits_names=[]
mean_gap=[]
median_gap=[]

#for every registered user find last edit timestamp, acount age, max inactive gap, number of edits timestamps
num=0
for name in registered['name']:
    num +=1
    print(num, name)
    # name='Ilmari Karonen'
    # convert registration date to datetime type
    registrasion_date=pd.to_datetime(registered.loc[registered['name']==name,'registration'], format='%Y-%m-%dT%H:%M:%SZ').to_frame()
    # print(registrasion_date.iloc[0,0])

    #load file with edit timestamps
    #create a list with the files of users in the username_edits directory
    onlyfiles = [f[:-4] for f in listdir('username_edits/') if isfile(join('/mnt/data/elisavetk/Wikidata_create_dataset/Create_csv_data_files/username_edits/', f))]

    #check if the registered user has edits
    if name in onlyfiles:
        f=open(str(directory_path)+str(name)+'.txt','r')
        data=f.readlines()

        data_df=pd.DataFrame({'timestamp':data})#convert the list of timestamps to dataframe
        dates=pd.to_datetime(data_df['timestamp'], format='%Y-%m-%dT%H:%M:%SZ\n').to_frame()#convert the strings to date time

        # check if there are edits with timestamp before 10/2012 and if yes, remove the timestamps and store the editors' names
        false_edits=dates[dates['timestamp']<'2012-09-30T00:00:00']
        if len(false_edits)>0:
            false_edits_names.append(name)
            # #find which rows have true, meaning which rows we want to extract
            # num_rows_lst=dates.index[dates['timestamp']<'2012-09-30T00:00:00'].tolist()
            #extract the rows from the edit hisotry data
            dates=dates.loc[~dates.index.isin(false_edits.index)]


        dates.sort_values(by='timestamp', ascending = True, inplace = True)# ascending order to date time
            
        #check if the editor has one or more edits
        if len(dates)>1: 
            date_diff=dates['timestamp'].diff()/ np.timedelta64(1, 'M')# find the time intervals in months between edits
            
            # store the data
            last_edit.append(dates.iloc[-1].values[0])
            age=(dates.iloc[-1]-registrasion_date.iloc[0,0])/ np.timedelta64(1, 'M')
            acount_age.append(age.values[0])
            max_gap.append(np.nanmax(date_diff.to_frame()))
            mean_gap.append(date_diff.to_frame().mean().values[0])
            median_gap.append(date_diff.to_frame().median().values[0])
            today_diff=(today-dates.iloc[-1])/ np.timedelta64(1, 'M')
            last_today.append(today_diff.values[0])
            number_of_edits.append(len(dates))
        else:
            #store the data
            last_edit.append(dates.iloc[-1].values[0])
            age=(dates.iloc[-1]-registrasion_date.iloc[0,0])/ np.timedelta64(1, 'M')
            acount_age.append(age.values[0])
            max_gap.append(0)
            mean_gap.append(0)
            median_gap.append(0)
            today_diff=(today-dates.iloc[-1])/ np.timedelta64(1, 'M')
            last_today.append(today_diff.values[0])
            number_of_edits.append(len(dates))
    else:
        last_edit.append(0)
        acount_age.append(0)
        max_gap.append(0)
        mean_gap.append(0)
        median_gap.append(0)
        today_diff=0
        last_today.append(0)
        number_of_edits.append(0)


#save the full data
my_data=pd.DataFrame({'last_edit':last_edit,'acount_age':acount_age,'max_gap':max_gap, 'mean_gap':mean_gap,'median_gap':median_gap,'last_today':last_today,'number_of_edits':number_of_edits})
save_data=pd.concat([registered,my_data.set_index(registered.index)], axis=1)
save_data.to_csv('username_info.csv', encoding='utf-8', index=False, header=True)

#save the false edits names
my_data_names=pd.DataFrame({'names':false_edits_names})
my_data_names.to_csv('false_edits.csv', encoding='utf-8', index=False, header=True)



### Status

We measured above the difference between the last edit and today for each editor. We set a thresfold at 23 months for active and inactive editors. This threshold was measured using the percentile of max gaps of inactivity 

In [ ]:
# Measure percentile

import pandas as pd
import numpy as np
df=pd.read_csv('username_stats.csv') # this includes the max_gap

status_threashold=np.percentile(df['max_gap'], 85)
status_threashold

In [ ]:
import pandas as pd

df=pd.read_csv('username_stats.csv')

status_threashold=23

status=[]
for last, num in zip(df.last_today, df.number_of_edits):
    if last<status_threashold and num!=0:
        status.append(1)# this is for active editors
    else:status.append(0)# this is for inactive editors

save_df=pd.concat([df,pd.DataFrame({'status':status})],axis=1)
save_df.to_csv('username_status.csv', encoding='utf-8')

### Access Level

We extracted and saved the usernames of the different access levels from the Wikidata interface https://www.wikidata.org/wiki/Special:ListUsers

In [ ]:
path='files_access_level_usernames/'

for file in os.listdir(path):
    if file[0]!='.':
        print(file)
        df_file=pd.read_csv(str(path)+str(file), encoding='utf-8')
        names_list=df_file['clean_name'].to_list()
        for name in df['name']:
            if name in names_list:
                label=df.loc[df['name']==name,'rights_label'].to_list()
                if 'editor' in label: label.remove('editor')
                label.append(file[:-4])
                # label=str(label)+';'+str(file[:-4])
                df.loc[df['name']==name,'rights_label']=';'.join(label)

df.to_csv('registered_editors_access_levels.csv', index=None,encoding='utf-8')

    

1=editor
14=acount creator
15=bot 
4=buraucrats
5=checkusers
6=confirmed users
7=flooders
8= interface administartors
9= IP exeptions
10=rollblockers
11=suppresors
12=translator administrators
13= wikidata staff
2= property creator
3= administrators

In [ ]:
import pandas as pd

df=pd.read_csv('registered_editors_access_levels.csv', encoding='utf-8')

data_lst=[]
for i in range(len(df)):
    labels=df.loc[i,'rights_label']
    all_labels=labels.split(';')
    for j in all_labels:
        if j == 'editor':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],1))
        elif j == 'acount_creator':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],14))
        elif j == 'administrators':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],3))
        elif j == 'bots':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],15))
        elif j == 'buraucrats':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],4))
        elif j == 'checkusers':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],5))
        elif j == 'confirmed_users':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],6))
        elif j == 'flooders':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],7))
        elif j == 'interface_administrators':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],8))
        elif j == 'IP_exeptions':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],9))
        elif j == 'property_creator':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],2))
        elif j == 'rollblockers':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],10))
        elif j == 'suppresors':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],11))
        elif j == 'translator_administrators':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],12))
        elif j == 'wikidata_staff':
            data_lst.append((df.loc[i,'name'],df.loc[i,'username_type'],df.loc[i,'acount_age'],df.loc[i,'number_of_edits'],df.loc[i,'total_posts'],df.loc[i,'talking_type'],df.loc[i,'status'],13))

df_rights=pd.DataFrame(data_lst, columns =['name','username_type','acount_age', 'number_of_edits', 'total_posts','talking_type','status','rights'])
df_rights.to_csv('registered_editors_coded_access_levels.csv',index=None, encoding='utf-8')


### Talking type

In [ ]:
import pandas as pd
import os
import itertools
import csv
import json

files_path='discussions/'# the directory where we store thw csv with all discussions separated into posts
save_path='asking_responding_channels/'# JSON files with all editors. Each has the number of times their post was first in a thread, and their posts was not first in the thread

data={} # to store the edges as tuples
#read files for the different discussion channels
for file in os.listdir(str(files_path)):

    df=pd.read_csv(str(files_path)+str(file), encoding='utf-8')# load data for the file
    subjects=df['thread_title'].unique() # keep using thread titles

    #create edges based on the thread title
    for subject in subjects:
        print(file, subject)
        if subject!='No subject': # if the thread has no title (i.e., 'No title') process based on the page title
            thread=df.loc[df['thread_title']==subject]# store the rows of posts from the same thread
            # for every name create a tuple with the other names in the thread
            thread.reset_index(drop=True, inplace=True)
            for row in range(len(thread)):
                if row==0:
                    if thread.loc[row,'urername'] in data.keys():
                        data[thread.loc[row,'urername']]=[data[thread.loc[row,'urername']][0]+1,data[thread.loc[row,'urername']][1]]
                    else:
                        data[thread.loc[row,'urername']]=[1,0]
                else:
                    if thread.loc[row,'urername'] in data.keys():
                        data[thread.loc[row,'urername']]=[data[thread.loc[row,'urername']][0],data[thread.loc[row,'urername']][1]+1]
                    else:
                        data[thread.loc[row,'urername']]=[0,1]
        else:
            no_title_rows=df.loc[df['thread_title']==subject]# store the posts with 'No title' as thread title
            pages=no_title_rows['page_title'].unique() # keep unique pages names
            for page in pages:
                print(file, subject, page)
                thread=df.loc[(df['page_title']==page) & (df['thread_title']==subject)]
                thread.reset_index(drop=True, inplace=True)
                for row in range(len(thread)):
                    if row==0:
                        if thread.loc[row,'urername'] in data.keys():
                            data[thread.loc[row,'urername']]=[data[thread.loc[row,'urername']][0]+1,data[thread.loc[row,'urername']][1]]
                        else:
                            data[thread.loc[row,'urername']]=[1,0]
                    else:
                        if thread.loc[row,'urername'] in data.keys():
                            data[thread.loc[row,'urername']]=[data[thread.loc[row,'urername']][0],data[thread.loc[row,'urername']][1]+1]
                        else:
                            data[thread.loc[row,'urername']]=[0,1]

    

    with open(str(save_path)+str(file[:-4])+'.json', 'w', encoding='utf-8') as fp:
        json.dump(data, fp,ensure_ascii=False)

    data={}

In [ ]:
import json
import os
import csv
import pandas as pd

def mergeDictionary(dict_1, dict_2):
   dict_3 = {**dict_1, **dict_2}
   for key, value in dict_3.items():
       if key in dict_1 and key in dict_2:
               dict_3[key] = [sum(x) for x in zip(value, dict_1[key])]
   return dict_3

files_path='asking_responding_channels/'

dict_3={} # to store the edges as tuples
#read files for the different discussion channels

for file in os.listdir(str(files_path)):
   # Opening JSON file
   f = open(str(files_path)+str(file), encoding='utf-8')
   data1 = json.load(f)
   dict_3 = mergeDictionary(dict_3, data1)

   # print('1: ',[d[0] for d in dict_3.values()])
   # print('2: ', [d[1] for d in dict_3.values()])
   # print([sum(x) for x in zip([d[0] for d in dict_3.values()], [d[1] for d in dict_3.values()] )])
df=pd.DataFrame({'name':dict_3.keys(),'asking':[d[0] for d in dict_3.values()], 'responding':[d[1] for d in dict_3.values()], 'total_posts':[sum(x) for x in zip([d[0] for d in dict_3.values()], [d[1] for d in dict_3.values()] )]})
df.to_csv('asking_responding.csv',encoding='utf-8', index=None)



In [ ]:
import pandas as pd

df=pd.read_csv('asking_responding.csv')

status_threashold=19

status=[]
for a, r in zip(df.asking, df.responding):
    if a>r:
        status.append(1)# this is asking 
    elif a<r:
        status.append(0)# this is for responding
    else: status.append(2)# this is for equal

save_df=pd.concat([df,pd.DataFrame({'talking_type':status})],axis=1)
save_df.to_csv('username_talking_type.csv', encoding='utf-8')